In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Cargar el archivo y eliminar filas donde el clúster es NaN
file_path = 'data/data.csv'
data = pd.read_csv(file_path)
data = data.dropna(subset=['Cluster_Label'])


2024-06-25 15:30:34.711898: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 15:30:34.715441: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-25 15:30:34.757966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 15:30:34.758005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 15:30:34.759217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Convertir 'year' y 'month_no' a una sola columna de tipo fecha
data['date'] = pd.to_datetime(data['year'].astype(str) + '-' + data['month_no'].astype(str))

# Seleccionar las columnas necesarias para la serie temporal
columns_to_keep = ['date', 'species', 'Cluster_Label', 'landed_w_kg', 'value_mxn',
                   '0.49402499_m', '1.541375_m', '2.645669_m', '3.819495_m', '5.0782242_m',
                   '6.4406142_m', '7.9295602_m', '9.5729971_m', 'mean_temp']
data = data[columns_to_keep]

# Agrupar por 'species' y 'Cluster_Label', y ordenar cada grupo por 'date'
grouped_data = data.groupby(['species', 'Cluster_Label'])
valid_series = {key: group for key, group in grouped_data if group['date'].nunique() >= 57}

# Extraer las series que cumplen el criterio
filtered_data = pd.concat(valid_series.values())

In [8]:
filtered_data= filtered_data[filtered_data['species']!='OTRAS']

In [9]:
filtered_data['species'].unique()

array(['ALMEJA', 'BANDERA', 'BAQUETA', 'BERRUGATA', 'CABRILLA', 'CALAMAR',
       'CARACOL', 'CAZON', 'CORVINA', 'GUACHINANGO', 'JAIBA', 'JUREL',
       'LENGUADO', 'LISA', 'LOBINA', 'MERO', 'MOJARRA', 'OSTION',
       'PAMPANO', 'PARGO', 'PIERNA', 'PULPO', 'RAYA Y SIMILARES',
       'ROBALO', 'RONCO', 'SARDINA', 'SIERRA', 'TIBURON'], dtype=object)

In [10]:
species_list = ['ALMEJA', 'BANDERA', 'BAQUETA', 'BERRUGATA', 'CABRILLA', 'CALAMAR',
                'CARACOL', 'CAZON', 'CORVINA', 'GUACHINANGO', 'JAIBA', 'JUREL',
                'LENGUADO', 'LISA', 'LOBINA', 'MERO', 'MOJARRA', 'OSTION',
                'PAMPANO', 'PARGO', 'PIERNA', 'PULPO', 'RAYA Y SIMILARES',
                'ROBALO', 'RONCO', 'SARDINA', 'SIERRA', 'TIBURON']

In [11]:
from sklearn.preprocessing import MinMaxScaler


# Función para preparar los datos
def prepare_data(group, look_back=3):
    features = ['landed_w_kg','Cluster_Label', '0.49402499_m', '1.541375_m', '2.645669_m', '3.819495_m', 
                '5.0782242_m', '6.4406142_m', '7.9295602_m', '9.5729971_m', 'mean_temp']
    scaler = MinMaxScaler(feature_range=(0, 1))
    group_scaled = scaler.fit_transform(group[features])
    
    X, y = [], []
    for i in range(len(group_scaled) - look_back):
        a = group_scaled[i:(i + look_back)]
        X.append(a)
        y.append(group_scaled[i + look_back, 0])  # La primera columna es 'landed_w_kg'
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    return X, y


In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import numpy as np


# Función para crear y entrenar un modelo LSTM
def create_and_train_lstm_model(data, species_name, epochs=50, batch_size=1):
    X, y = prepare_data(data)
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(25, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(12, activation='linear'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    #checkpoint_path = f'modelos/{species_name}_best_model.h5'
    #checkpoint = ModelCheckpoint(checkpoint_path, monitor='loss', save_best_only=True, mode='min', verbose=1)

    model.fit(X, y,validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])
    return model

In [19]:
models = {}
for species in species_list:
    print(f'Entrenamiento de {species}')
    species_data = data[data['species'] == species]  # Filtrar datos por especie
    models[species] = create_and_train_lstm_model(species_data, species)

Entrenamiento de ALMEJA
Epoch 1/50
14977/14977 [==============================] - 50s 3ms/step - loss: 4.3941e-04 - val_loss: 5.4780e-04
Epoch 2/50
14977/14977 [==============================] - 46s 3ms/step - loss: 4.1727e-04 - val_loss: 4.5006e-04
Epoch 3/50
14977/14977 [==============================] - 46s 3ms/step - loss: 3.7055e-04 - val_loss: 4.8162e-04
Epoch 4/50
14977/14977 [==============================] - 46s 3ms/step - loss: 3.6324e-04 - val_loss: 4.9698e-04
Epoch 5/50
14977/14977 [==============================] - 46s 3ms/step - loss: 3.6180e-04 - val_loss: 4.8192e-04
Epoch 6/50
14977/14977 [==============================] - 46s 3ms/step - loss: 3.5788e-04 - val_loss: 5.0713e-04
Epoch 7/50
14977/14977 [==============================] - 46s 3ms/step - loss: 3.5799e-04 - val_loss: 4.5896e-04
Epoch 8/50
14977/14977 [==============================] - 46s 3ms/step - loss: 3.5641e-04 - val_loss: 5.1868e-04
Epoch 9/50
14977/14977 [==============================] - 46s 3ms/step -

In [ ]:
from keras.models import load_model
# Asumimos que todas las funciones necesarias y los modelos están cargados
def moe_predict(species_name, X):
    model_path = f'{species_name}_best_model.h5'
    model = load_model(model_path)
    prediction = model.predict(X)
    return prediction


In [ ]:
# Uso 
species_input = 'ALMEJA'
#X_input 
result = moe_predict(species_input, X_input)